In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from workflow.scripts.plotting_tools import get_model_colordict

def get_forcing(forcing_var: str,dataframes: dict):
    k = next(iter(dataframes))
    outdf = pd.DataFrame(index=dataframes.keys(), columns=dataframes[k].columns)
    for k,df in dataframes.items():
        try:
            outdf.loc[k,:] = df.loc[forcing_var]
        except KeyError:
            pass
            
    return outdf

dfs_dusty = {p.split('.')[0].split('_')[-1]: pd.read_csv(p,index_col=0) for p in snakemake.input.dusty_region}
dfs_all = {p.split('.')[0].split('_')[-1]: pd.read_csv(p,index_col=0) for p in snakemake.input.all_region}
dfs_nondusty = {p.split('.')[0].split('_')[-1]: pd.read_csv(p,index_col=0) for p in snakemake.input.nodusty_region}
colors = get_model_colordict(2)

In [ ]:

erf_dusty = get_forcing('ERFt', dfs_dusty)
erf_notdust = get_forcing('ERFt', dfs_nondusty)
erf_all = get_forcing('ERFt', dfs_all)

fig,ax = plt.subplots(figsize=(4,4))
# plt.axhline(0,0 ,linestyle=':', linewidth=2, color='darkgrey')
plt.axvline(0,0 ,linestyle=':', linewidth=2, color='darkgrey')
for du,nondu, duall in zip(erf_dusty['diff'].items(),erf_notdust['diff'], erf_all['diff']): 
    ax.scatter(duall,abs(du[1])/(abs(nondu)+abs(du[1])), label=du[0], color=colors[du[0]], zorder=1001)
# erf_dusty.plot.scatter(y=erf_notdust['diff'])
# plt.legend()
# ax.set_xlim(-.2,.2)
# ax.set_ylim(-1,4)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_xlabel('Dust ERF in dusty regions \n [w m-2]')
ax.set_ylabel('Non-dusty/ \n [w m-2]')

In [ ]:
forcings_dusty = {
  'ERFt' : get_forcing('ERFt', dfs_dusty),
  'ERFsuf' : get_forcing('ERFsurf', dfs_dusty),
  'SWCloudEff' : get_forcing('SWCloudEff', dfs_dusty),
  'LWCloudEff' : get_forcing('LWCloudEff', dfs_dusty),
  'SWDirectEff' : get_forcing('SWDirectEff', dfs_dusty),
  'LWDirectEff' : get_forcing('LWDirectEff', dfs_dusty),
}

forcings_nondusty = [
    get_forcing('ERFt', dfs_nondusty),
    get_forcing('ERFsurf', dfs_nondusty),
    get_forcing('SWCloudEff', dfs_nondusty),
    get_forcing('LWCloudEff', dfs_nondusty),
    get_forcing('SWDirectEff', dfs_nondusty),
    get_forcing('LWDirectEff', dfs_nondusty),
]


In [ ]:
fig,axes = plt.subplots(ncols=6, figsize=(10,4))
for ax in axes:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_ylim(0,10)
    ax.spines['left'].set_visible(False)
    ax.set_yticks([])
    ax.vlines(0.0, 0,10, color='darkgrey', linestyle=':')
xticks = [i for i in range(1,10)]
axes[0].set_yticks([i for i in range(1,10)])
axes[0].set_yticklabels(erf_all.index)
for ax, duf, noduf, vname in zip(axes, forcings_dusty.values(), forcings_nondusty, forcings_dusty):
    for i,nondu, du in zip(xticks,noduf.iterrows(), duf.iterrows()):
        ax.barh(y=i-.125,width=nondu[1]['diff'], height=.25, color='tab:orange')
        ax.barh(y=i+.125,width=du[1]['diff'], height=.25, color='tab:blue')
    ax.set_xlabel(vname)
    
plt.savefig(snakemake.output.outpath,bbox_inches='tight')
plt.savefig('results/figs/AerChemMIP/dusty_vs_non_dusty.pdf', bbox_inches='tight')